In [ ]:
# from model.model import build_model
from model.calibration import get_calibration_bcm
from estival.wrappers.nevergrad import optimize_model

In [ ]:
fixed_params = {
    "time_start": 1800,
    "time_end": 2040,
    "time_step": 1,

    "start_population": 1000000,
    "crude_birth_rate": 17. / 1000.,

    "early_activation_rate": {
        "0": 2.4107,
        "5": 0.9862,
        "15": 0.0986
    },
    "late_activation_rate": {
        "0": 6.939769e-09,
        "5": 0.0023,
        "15": 0.0012
    },
    "stabilisation_rate":{
        "0": 4.383,
        "5": 4.383,
        "15": 1.972
    },

    "self_recovery_rate": 0.16,
    "tb_mortality_rate": 0.130,
    "background_mortality_rate":{  # Vietnam in 2018, extracted using get_death_rates_by_agegroup from https://github.com/monash-emu/AuTuMN/blob/master/autumn/core/inputs/demography/queries.py 
        '0': 0.0043347634430972014,
        '5': 0.0005100562239187423,
        '15': 0.0013469744398241523,
        '40': 0.005814942133889755,
        '60': 0.025162329043120622,
        '75': 0.14380537779630256,
    },

    "rr_reinfection": .20,

    "CDR_2000": 0.60,
    "TSR": 0.85,

    "use_interventions": False,
    "intervention_time": 2025,
    "max_transmission_reduction": .20,
    "max_pt_rate": .10,
    "max_intervention_cdr": .95

}

In [ ]:
bcm = get_calibration_bcm(fixed_params, target_incidence=200)

In [ ]:
orunner = optimize_model(bcm, num_workers=8)
rec = orunner.minimize(1000)
mle_params = rec.value[1]

In [ ]:
mle_params

In [ ]:
baseline_decision_vars = {
    "decision_var_trans": 0.,
    "decision_var_cdr": 0.,
    "decision_var_pt": 0.
}

In [ ]:
res = bcm.run(mle_params | baseline_decision_vars )
derived_df = res.derived_outputs

In [ ]:
derived_df['incidence_per100k'].plot()

## OPTIMISATION

In [ ]:
from model.optimisation import get_optimisation_bcm

In [ ]:
interv_params = fixed_params | {"use_interventions":True, "fitted_effective_contact_rate": mle_params["effective_contact_rate"]}

In [ ]:
o_bcm = get_optimisation_bcm(interv_params, decision_var_sum_threshold=1.5)

In [ ]:
o_res = o_bcm.run(baseline_decision_vars)
o_derived_df = o_res.derived_outputs
o_derived_df['incidence_per100k'].plot()

In [ ]:
fullon_decision_vars =  {
    "decision_var_trans": 1.,
    "decision_var_cdr": 1.,
    "decision_var_pt": 1.
}

In [ ]:
fullon_res = o_bcm.run(fullon_decision_vars)
fullon_derived_df = fullon_res.derived_outputs
fullon_derived_df['incidence_per100k'].plot()